<a href="https://www.comet.com/site/?utm_medium=colab&utm_source=comet-examples&utm_campaign=torchtune" >
  <img src="https://cdn.comet.ml/img/notebook_logo.png">
</a>

# Comet and torchtune

[Comet](https://www.comet.com/site/?utm_medium=colab&utm_source=comet-examples&utm_campaign=torchtune) is an MLOps platform designed to help data scientists and teams build better models faster! Comet provides tooling to track, explain, manage, and monitor your models in a single place! It works with Jupyter notebooks and scripts and-- most importantly--it's 100% free to get started!

[torchtune](https://github.com/pytorch/torchtune) is a PyTorch-native library for easily authoring, fine-tuning and experimenting with LLMs.

Instrument your torchtune training runs with Comet to start managing experiments with efficiency, reproducibility, and collaboration in mind.

Find more information about [our integration with torchtune here](https://www.comet.com/docs/v2/integrations/third-party-tools/torchtune?utm_medium=colab&utm_source=comet-examples&utm_campaign=torchtune) or [learn about our other integrations here](https://www.comet.com/docs/v2/integrations?utm_medium=colab&utm_source=comet-examples&utm_campaign=torchtune).


<a href="https://www.comet.com/examples/comet-examples-torchtune-mistral7b?utm_medium=colab&utm_source=comet-examples&utm_campaign=torchtune" >
  <img src="https://raw.githubusercontent.com/anmorgan24/Comet_Projects/main/Project%20Images/torchtune_and_comet.png">
</a>


🌟 Can't wait? For a preview for what's to come, check out the completed Comet project [here](https://www.comet.com/examples/comet-examples-torchtune-mistral7b?utm_medium=colab&utm_source=comet-examples&utm_campaign=torchtune).

In [ ]:
!git clone https://github.com/pytorch/torchtune/

## Install dependencies

In [ ]:
%pip install "./torchtune[dev]" comet_ml

## Login to Comet
For CLI, use `comet login`

In [ ]:
import comet_ml

comet_ml.login()

## Download the model

You need to login to HF to download that model:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!tune download mistralai/Mistral-7B-v0.1 \
    --output-dir /tmp/Mistral-7B-v0.1/

## Create the config file

You are now ready to fine-tune Mistral 7B model and logs your results to Comet. To log data to Comet, the important part is:

```yaml
metric_logger:
  _component_: torchtune.utils.metric_logging.CometLogger
```

In [ ]:
%%writefile mistral_comet_lora.yaml
tokenizer:
  _component_: torchtune.models.mistral.mistral_tokenizer
  path: /tmp/Mistral-7B-v0.1/tokenizer.model

# Dataset
dataset:
  _component_: torchtune.datasets.alpaca_cleaned_dataset
  train_on_input: True
seed: null
shuffle: True

# Model Arguments
model:
  _component_: torchtune.models.mistral.lora_mistral_7b
  lora_attn_modules: ['q_proj', 'k_proj', 'v_proj']
  apply_lora_to_mlp: True
  apply_lora_to_output: True
  lora_rank: 64
  lora_alpha: 16

checkpointer:
  _component_: torchtune.training.FullModelHFCheckpointer
  checkpoint_dir: /tmp/Mistral-7B-v0.1
  checkpoint_files: [
    pytorch_model-00001-of-00002.bin,
    pytorch_model-00002-of-00002.bin
  ]
  recipe_checkpoint: null
  output_dir: /tmp/Mistral-7B-v0.1
  model_type: MISTRAL
resume_from_checkpoint: False

optimizer:
  _component_: torch.optim.AdamW
  lr: 2e-5

lr_scheduler:
  _component_: torchtune.modules.get_cosine_schedule_with_warmup
  num_warmup_steps: 100

loss:
  _component_: torch.nn.CrossEntropyLoss

# Fine-tuning arguments
batch_size: 4
epochs: 1
max_steps_per_epoch: 100
gradient_accumulation_steps: 2
compile: False

# Training env
device: cuda

# Memory management
enable_activation_checkpointing: True

# Reduced precision
dtype: bf16
############################### Enable Comet ###################################
################################################################################
# Logging
# enable logging to the built-in CometLogger
metric_logger:
  _component_: torchtune.training.metric_logging.CometLogger
  # the Comet project to log to
  project: comet-examples-torchtune-mistral7b
  experiment_name: mistral7b-alpaca-cleaned
################################################################################
################################################################################
output_dir: /tmp/Mistral-7B-v0.1
log_peak_memory_stats: True

# Profiler (disabled)
profiler:
  _component_: torchtune.training.setup_torch_profiler
  enabled: False

## Train the model

In [ ]:
!tune run lora_finetune_single_device --config mistral_comet_lora.yaml

🌟 Check out the [final public project here](https://www.comet.com/examples/comet-examples-torchtune-mistral7b)!